# Descriptive statistics

In [ ]:
import numpy as np
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import statsmodels.api as sm
import statistics
import os 
from scipy.stats import norm

## Probability data, binomial distribution

We already got to know data that follow a binomial distribution, but we actually had not looked at the distribution. We will do this now. 10% of the 100 cells we count have deformed nuclei. To illustrate the distribution we will count repeatedly....  

In [ ]:
n = 100 # number of trials
p = 0.1 # probability of each trial

s = np.random.binomial(n, p, 1000) #simulation repeating the experiment 1000 times
print(s)

As you can see, the result of the distribution is in absolute counts, not proportions - they can easyly be converted by deviding with n, but they dont have to...  

In [ ]:
props = s/n
print(props)

Now we plot the distribution. The easiest first look is a histogram. 

In [ ]:
plt.hist(props, bins = 50)
plt.xlabel("proportion")
plt.ylabel("frequency")
plt.show()

The resolution is a bit inappropriate, given that we deal with integers. To increase the bin number would be a good idea. Maybe we should also plot a confidence interval.

In [ ]:
CI= sm.stats.proportion_confint(n*p, n, alpha=0.05)
print(CI)

plt.axvspan(CI[0],CI[1], alpha=0.2, color='yellow')
plt.hist(props, bins = 50)
plt.xlabel("proportion")
plt.ylabel("frequency")
plt.axvline(p, color="black")

In a binomial distribution, the distribution is given by the proportion and the sample size. Therefore we could calculate a confidence interval from one measurement.    

#### How can we now describe the distribution? 
Summary statistics:

In [ ]:
print("the minimum is:",  min(props))
print("the maximum is:",  max(props))

In [ ]:
print(statistics.mean(props))

Is the mean a good way to look at our distribution? 

In [ ]:
n = 50 # number of trials
p = 0.02 # probability of each trial

s = np.random.binomial(n, p, 1000) #simulation repeating the experiment 1000 times
props = s/n

CI= sm.stats.proportion_confint(n*p, n, alpha=0.05)
print(CI)

plt.axvspan(CI[0],CI[1], alpha=0.2, color='yellow')
plt.hist(props, bins = 20)
plt.xlabel("proportion")
plt.ylabel("frequency")
plt.axvline(p, color="black")
plt.axvline(statistics.mean(props), color="red")

print(statistics.mean(props))

In [ ]:
n = 500 # number of trials
p = 0.02 # probability of each trial

s = np.random.binomial(n, p, 1000) #simulation repeating the experiment 1000 times
props = s/n

CI= sm.stats.proportion_confint(n*p, n, alpha=0.05)
print(CI)

plt.axvspan(CI[0],CI[1], alpha=0.2, color='yellow')
plt.hist(props, bins = 50)
plt.xlabel("proportion")
plt.ylabel("frequency")
plt.axvline(p, color="black")
plt.axvline(statistics.mean(props), color="red")

print(statistics.mean(props))

## Count data/ the Poisson distribution

The Poisson distribution is built on count data, e.g. the numbers of raisins in a Dresdner Christstollen, the number of geese at any given day between Blaues Wunder and Waldschlösschenbrücke, or radioactive decay. So lets use a Geiger counter and count the numbers of decay per min.  

In [ ]:
freq =1.6
s = np.random.poisson(freq, 1000)

plt.hist(s, bins = 20)
plt.xlabel("counts per minute")
plt.ylabel("frequency")
plt.axvline(freq, color="black")

### Confidence intervals for a Poisson distribution

Similar to the binomial distribution, the distribution is defined by sample size and the mean.  
Also for Poisson, one can calculate an also asymmetrical confidence interval:

In [ ]:
freq =1.6
s = np.random.poisson(freq, 1000)
CI = st.poisson.interval(0.95,freq)

plt.axvspan(CI[0],CI[1], alpha=0.2, color='yellow')
plt.hist(s, bins = 20)
plt.xlabel("counts per minute")
plt.ylabel("frequency")
plt.axvline(freq, color="black")


For a poisson distribution, poisson error can be reduced, when increasing the counting population, in our case lets count for 10 min instead of 1 min, and see what happens.   

In [ ]:
CI = np.true_divide(st.poisson.interval(0.95,freq*10),10)
print(CI)
s = np.true_divide(np.random.poisson(freq*10, 1000),10)

plt.axvspan(CI[0],CI[1], alpha=0.2, color='yellow')
plt.hist(s, bins = 70)
plt.xlabel("counts per minute")
plt.ylabel("frequency")
plt.axvline(freq, color="black")

What is the difference between Poisson and Binomial? Aren't they both kind of looking at count data?   
Yes, BUT:   
Binomial counts events versus another event, e.g. for the cells there are two options, normal versus deformed. A binomial distribution is about comparing the two options.   
Poisson counts with an open end, e.g. number of mutations.  

## Continuous data

Let's import the count data you have generated with Robert. When you download it from Google sheets (https://docs.google.com/spreadsheets/d/1Ek-23Soro5XZ3y1kJHpvaTaa1f4n2C7G3WX0qddD-78/edit#gid=0), it comes with spaces. Try to avoid spaces and special characters in file names, they tend to make trouble.   
I renamed it to 'BBBC001.csv'.

In [ ]:
dat = pd.read_csv('https://raw.githubusercontent.com/BiAPoL/Bio-image_Analysis_with_Python/main/data/BBBC001.csv', header=1, sep=';')

print(dat)

For now we will focus on the manual counts, visualise it and perform summary statistics.

In [ ]:
man_count = dat["BBBC001 manual count"].values
auto_count = dat.iloc[:,[2,3,4]].values

plt.hist(man_count,bins=100)

In [ ]:
print(man_count)

In [ ]:
plt.hist(auto_count,bins=100)

In [ ]:
sns.kdeplot(data=dat)

There are different alternatives of displaying such data, some of which independent of distribution. You will find documentation in the graph galery: https://www.python-graph-gallery.com/

In [ ]:
sns.kdeplot(man_count)

A density plot is sometimes helpful to see the distribution, but be aware of the smoothing and that you loose the information on sample size. 

In [ ]:
sns.stripplot(data=auto_count)

In [ ]:
sns.swarmplot(y=man_count)

In [ ]:
sns.violinplot(y=man_count)

this plot is useful, but the density function can sometimes be misleading and lead to artefacts dependent on the sample size. Unless explicitely stated, sample sizes are usually normalised and therefore hidden!

In [ ]:
sns.boxplot(y=man_count)

Be aware that boxplots hide the underlying distribution and the sample size.  
So the "safest" plot, when in doubt, is to combine boxplot and jitter:

In [ ]:
ax = sns.swarmplot(y=man_count, color="black")
ax = sns.boxplot(y=man_count,color="lightgrey")
ax.grid()

The boxplot is very useful, because it directly provides non-parametric summary statistics:  
Min, Max, Median, Quartiles and therefore the inter-quartile range (IQR). The whiskers are usually the highest point that is within 1.5x the quartile plus the IQR. Everything beyond that is considered an outlier. Whiskers are however not always used in this way! 
The mean and standard diviation are not visible in a boxplot, because it is only meaningful in distributions that center around the mean. It is however a part of summary statistics:

In [ ]:
dat["BBBC001 manual count"].describe()

## Normal distribution

We assume that our distribution is "normal".  
First we fit a normal distribution to our data. 

In [ ]:
(mu, sigma) = norm.fit(man_count)
n, bins, patches = plt.hist(man_count, 100,density=1)


# add a 'best fit' line
y = norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

#plot
plt.xlabel('manual counts counts')
plt.ylabel('binned counts')
plt.title(r'$\mathrm{Histogram\ of\ manual\ counts:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))

plt.show()

Is it really normally distributed? What we see here is already one of the most problematic properties of a normal distribution: The susceptibility to outliers. 

In normal distributions the confidence interval is determined by the standard diviation. A confidence level of 95% equals 1.96 x sigma.  

In [ ]:
#plot

(mu, sigma) = norm.fit(man_count)
n, bins, patches = plt.hist(man_count, 100,density=1)

# add a 'best fit' line
y = norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)


plt.xlabel('manual counts counts')
plt.ylabel('binned counts')
plt.title(r'$\mathrm{Histogram\ of\ manual\ counts:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
plt.axvspan((mu-1.96*sigma),(mu+1.96*sigma), alpha=0.2, color='yellow')
plt.axvline(mu, color="black")


plt.show()

This shows even nicer that our outlier messes up the distribution :-)  
How can we solve this in practise?  
1. Ignore the problem and continue with the knowledge that we are overestimating the width of the distribution and underestimating the mean.  
2. Censor the outlier. 
3. Decide that we cannot assume normality and move to either a different distribution or non-parametric statistics. 

## Other distributions   
Of course there are many more distributions, e.g.  
Lognormal is a distribution that becomes normal, when log transformed. It is important for the "geometric mean".  
Bimodal distributions may arise from imaging data with background signal, or DNA methylation data.   
Negative binomial distributions are very important in genomics, especially RNA-Seq analysis. 

## Exercise

1. We had imported the total table with also the automated counts. Visualise the distribution of the data next to each other
2. Generate the summary statistics and compare the different distributions
3. Two weeks ago you learned how to analyze a folder of images and measured the average size of beads:
https://nbviewer.jupyter.org/github/BiAPoL/Bio-image_Analysis_with_Python/blob/main/image_processing/12_process_folders.ipynb

Go back to the bead-analysis two weeks ago and measure the intensity of the individual beads (do not average over the image). Plot the beads' intensities as different plots. Which one do you find most approproiate for these data?

